# YOLOv11 - Detección de Autos en Video

In [38]:
import sys
import platform

# ============================================
# CELDA 1: Verificar sistema macOS
# ============================================
print("🍎 Verificando sistema macOS...")
print(f"Sistema: {platform.system()}")
print(f"Versión: {platform.mac_ver()[0]}")
print(f"Procesador: {platform.processor()}")
print(f"Arquitectura: {platform.machine()}")

# Detectar si es Apple Silicon
is_apple_silicon = platform.machine() == 'arm64'
if is_apple_silicon:
    print("✅ Apple Silicon detectado (M1/M2/M3)")
else:
    print("✅ Intel Mac detectado")

🍎 Verificando sistema macOS...
Sistema: Darwin
Versión: 26.0.1
Procesador: arm
Arquitectura: arm64
✅ Apple Silicon detectado (M1/M2/M3)


In [39]:
# ============================================
# CELDA 2: Instalar dependencias
# ============================================
%pip install ultralytics opencv-python torch torchvision torchaudio gdown

Note: you may need to restart the kernel to use updated packages.


In [40]:
# ============================================
# CELDA 3: Importar librerías
# ============================================
from ultralytics import YOLO
import cv2
import numpy as np
from pathlib import Path
import torch
import os
import gdown

print("\n✅ Librerías importadas correctamente")
print(f"\n🔧 Información del sistema:")
print(f"   Python: {sys.version.split()[0]}")
print(f"   PyTorch: {torch.__version__}")

# Detectar aceleración disponible en macOS
if torch.backends.mps.is_available():
    device = 'mps'  # Metal Performance Shaders (Apple Silicon)
    print(f"   🚀 Aceleración: MPS (Metal) - Apple Silicon")
elif torch.cuda.is_available():
    device = 'cuda'
    print(f"   🚀 Aceleración: CUDA")
else:
    device = 'cpu'
    print(f"   💻 Aceleración: CPU")

print(f"   Dispositivo seleccionado: {device}")


✅ Librerías importadas correctamente

🔧 Información del sistema:
   Python: 3.11.13
   PyTorch: 2.9.0
   🚀 Aceleración: MPS (Metal) - Apple Silicon
   Dispositivo seleccionado: mps


In [ ]:
# ============================================
# CELDA 4: Cargar modelo
# ============================================
# Modelos disponibles: 'n' (nano), 's' (small), 'm' (medium), 'l' (large), 'x' (xlarge)
MODEL_SIZE = 'n'

print(f"\n🤖 Cargando modelo YOLOv11-{MODEL_SIZE}...")
model = YOLO(f'yolo11{MODEL_SIZE}.pt')

# Configurar dispositivo
if device != 'cpu':
    model.to(device)
    print(f"✅ Modelo cargado en {device.upper()}")
else:
    print(f"✅ Modelo cargado en CPU")


🤖 Cargando modelo YOLOv11-n...
✅ Modelo cargado en MPS


In [42]:
# ============================================
# CELDA 5: Función para descargar de Google Drive
# ============================================
def descargar_desde_google_drive(url_compartido, nombre_salida='video_descargado.mp4'):
    """
    Descarga un video desde Google Drive usando un link compartido
    
    Args:
        url_compartido: URL de Google Drive completa
        nombre_salida: Nombre del archivo local
    
    Returns:
        str: Ruta del archivo descargado
    """
    print(f"\n📥 Descargando video desde Google Drive...")
    print(f"   URL: {url_compartido[:50]}...")
    
    # Extraer el ID del archivo
    file_id = None
    
    if '/file/d/' in url_compartido:
        file_id = url_compartido.split('/file/d/')[1].split('/')[0]
    elif 'open?id=' in url_compartido:
        file_id = url_compartido.split('open?id=')[1].split('&')[0]
    elif 'uc?id=' in url_compartido:
        file_id = url_compartido.split('uc?id=')[1].split('&')[0]
    elif 'id=' in url_compartido:
        file_id = url_compartido.split('id=')[1].split('&')[0]
    elif len(url_compartido) < 100 and '/' not in url_compartido:
        file_id = url_compartido
    
    if not file_id:
        print("❌ No se pudo extraer el ID del archivo")
        print("💡 Formato esperado: https://drive.google.com/file/d/FILE_ID/view?usp=sharing")
        return None
    
    print(f"   File ID: {file_id}")
    
    try:
        # Construir URL de descarga
        download_url = f'https://drive.google.com/uc?id={file_id}'
        
        # Descargar
        print(f"   Descargando a: {nombre_salida}")
        print(f"   Por favor espera, esto puede tardar varios minutos...")
        
        gdown.download(download_url, nombre_salida, quiet=False, fuzzy=True)
        
        if Path(nombre_salida).exists():
            tamanio_mb = Path(nombre_salida).stat().st_size / (1024 * 1024)
            print(f"\n✅ Descarga completa!")
            print(f"   Archivo: {nombre_salida}")
            print(f"   Tamaño: {tamanio_mb:.2f} MB")
            return nombre_salida
        else:
            print("❌ Error: El archivo no se descargó correctamente")
            return None
            
    except Exception as e:
        print(f"❌ Error al descargar: {e}")
        print("\n💡 Soluciones:")
        print("   1. Verifica que el link sea público ('Cualquiera con el enlace')")
        print("   2. Intenta con un video más pequeño primero")
        print("   3. Asegúrate de tener espacio en disco")
        return None

print("✅ Función de descarga de Google Drive lista")

✅ Función de descarga de Google Drive lista


In [ ]:
# ============================================
# CELDA 6: CONFIGURACIÓN - ¡AQUÍ ES DONDE CONFIGURAS TODO!
# ============================================

USAR_GOOGLE_DRIVE = True  # ⬅️ True = Google Drive, False = archivo local

if USAR_GOOGLE_DRIVE:
    GOOGLE_DRIVE_URL = 'https://drive.google.com/file/d/1QHZZJ7EHW14AXu0QVr4k21wftG1dP4DZ/view?usp=drive_link'
    
    print(f"\n{'='*60}")
    print("📥 MODO: Descarga desde Google Drive")
    print(f"{'='*60}")
    
    # Descargar el video
    VIDEO_PATH = descargar_desde_google_drive(
        url_compartido=GOOGLE_DRIVE_URL,
        nombre_salida='video_desde_drive.mp4'
    )
    
    if not VIDEO_PATH:
        print("\n⚠️ No se pudo descargar. Revisa el link de Google Drive.")
        print("💡 Asegúrate de:")
        print("   1. El video está en 'Cualquiera con el enlace'")
        print("   2. El link está correcto")
        print("   3. Tienes internet estable")
        sys.exit(1)

else:
    # 🔴 PASO 3: Si usas archivo local, pon el nombre aquí:
    VIDEO_PATH = 'tu_video.mp4'  # ⬅️ CAMBIA ESTO
    
    print(f"\n{'='*60}")
    print("📁 MODO: Archivo local")
    print(f"{'='*60}")
    
    if not Path(VIDEO_PATH).exists():
        print(f"❌ Error: No se encontró '{VIDEO_PATH}'")
        print("💡 Coloca el video en la misma carpeta que este script")
        sys.exit(1)

# Configuración adicional
OUTPUT_PATH = 'video_con_detecciones.mp4'  # Nombre del video de salida
CONF_THRESHOLD = 0.4  # Umbral de confianza (0.0 - 1.0)

# Verificar el video
if VIDEO_PATH and Path(VIDEO_PATH).exists():
    print(f"\n✅ Video cargado: {VIDEO_PATH}")
    
    cap_test = cv2.VideoCapture(VIDEO_PATH)
    if cap_test.isOpened():
        width = int(cap_test.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap_test.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap_test.get(cv2.CAP_PROP_FPS))
        frames = int(cap_test.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = frames / fps if fps > 0 else 0
        
        print(f"\n📊 Información del video:")
        print(f"   📐 Resolución: {width}x{height}")
        print(f"   🎞️  FPS: {fps}")
        print(f"   📹 Frames totales: {frames}")
        print(f"   ⏱️  Duración: {duration:.2f} segundos ({duration/60:.2f} minutos)")
        
        tamanio_mb = Path(VIDEO_PATH).stat().st_size / (1024 * 1024)
        print(f"   💾 Tamaño: {tamanio_mb:.2f} MB")
        
        cap_test.release()
        
        print(f"\n⚙️ Configuración:")
        print(f"   Modelo: YOLOv11-{MODEL_SIZE}")
        print(f"   Dispositivo: {device.upper()}")
        print(f"   Umbral: {CONF_THRESHOLD}")
        print(f"   Salida: {OUTPUT_PATH}")
    else:
        print("❌ Error: No se pudo abrir el video")
        sys.exit(1)


📥 MODO: Descarga desde Google Drive

📥 Descargando video desde Google Drive...
   URL: https://drive.google.com/file/d/1QHZZJ7EHW14AXu0QV...
   File ID: 1QHZZJ7EHW14AXu0QVr4k21wftG1dP4DZ
   Descargando a: video_desde_drive.mp4
   Por favor espera, esto puede tardar varios minutos...


Downloading...
From (original): https://drive.google.com/uc?id=1QHZZJ7EHW14AXu0QVr4k21wftG1dP4DZ
From (redirected): https://drive.google.com/uc?id=1QHZZJ7EHW14AXu0QVr4k21wftG1dP4DZ&confirm=t&uuid=d51ab486-a521-417e-8a63-638dee4c54b2
To: /Users/abrahamsolorzanopenaloza/vision-example/video_desde_drive.mp4
100%|██████████| 184M/184M [00:02<00:00, 75.9MB/s] 


✅ Descarga completa!
   Archivo: video_desde_drive.mp4
   Tamaño: 175.33 MB

✅ Video cargado: video_desde_drive.mp4

📊 Información del video:
   📐 Resolución: 3840x2160
   🎞️  FPS: 30
   📹 Frames totales: 1800
   ⏱️  Duración: 60.00 segundos (1.00 minutos)
   💾 Tamaño: 175.33 MB

⚙️ Configuración:
   Modelo: YOLOv11-n
   Dispositivo: MPS
   Umbral: 0.4
   Salida: video_con_detecciones.mp4


In [44]:
# ============================================
# CELDA 7: Función de detección
# ============================================
def detect_vehicles_macos(video_path, output_path, conf_threshold=0.4, device='cpu'):
    """Detecta vehículos en video - Optimizado para macOS"""
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ No se pudo abrir el video")
        return None
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"\n{'='*60}")
    print(f"🎬 INICIANDO PROCESAMIENTO")
    print(f"{'='*60}")
    
    # Codec para macOS
    if sys.platform == 'darwin':
        fourcc = cv2.VideoWriter_fourcc(*'avc1')  # H.264
    else:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    if not out.isOpened():
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Clases de vehículos
    vehicle_classes = {2: 'Auto', 3: 'Moto', 5: 'Bus', 7: 'Camión'}
    colors = {2: (0, 255, 0), 3: (255, 0, 0), 5: (0, 255, 255), 7: (255, 0, 255)}
    
    frame_count = 0
    max_vehicles = 0
    stats = {cls: 0 for cls in vehicle_classes.keys()}
    
    print(f"\n⏳ Procesando {total_frames} frames...")
    print(f"💡 Esto puede tardar varios minutos\n")
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            
            # Detección
            results = model(frame, conf=conf_threshold, verbose=False, device=device)
            
            vehicle_counts = {cls: 0 for cls in vehicle_classes.keys()}
            
            # Procesar detecciones
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    cls = int(box.cls[0])
                    
                    if cls in vehicle_classes:
                        vehicle_counts[cls] += 1
                        stats[cls] = max(stats[cls], vehicle_counts[cls])
                        
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        conf = float(box.conf[0])
                        
                        color = colors[cls]
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        
                        label = f'{vehicle_classes[cls]} {conf:.2f}'
                        label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                        cv2.rectangle(frame, (x1, y1 - label_size[1] - 10), 
                                    (x1 + label_size[0], y1), color, -1)
                        cv2.putText(frame, label, (x1, y1 - 5),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
            
            # Panel de estadísticas
            total_vehicles = sum(vehicle_counts.values())
            max_vehicles = max(max_vehicles, total_vehicles)
            
            active_classes = [cls for cls in vehicle_classes.keys() if vehicle_counts[cls] > 0]
            panel_height = 40 + len(active_classes) * 25
            
            cv2.rectangle(frame, (10, 10), (260, panel_height), (0, 0, 0), -1)
            cv2.rectangle(frame, (10, 10), (260, panel_height), (255, 255, 255), 2)
            
            y_pos = 35
            cv2.putText(frame, f'Total Vehiculos: {total_vehicles}', (20, y_pos),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            y_pos += 30
            for cls in active_classes:
                name = vehicle_classes[cls]
                cv2.putText(frame, f'{name}: {vehicle_counts[cls]}', 
                          (20, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                          colors[cls], 2)
                y_pos += 25
            
            out.write(frame)
            
            # Progreso cada 30 frames
            if frame_count % 30 == 0:
                progress = (frame_count / total_frames) * 100
                tiempo_restante = ((total_frames - frame_count) / 30) * 1  # estimación
                print(f"⏳ {progress:.1f}% [{frame_count}/{total_frames}] - Vehículos: {total_vehicles} - ~{tiempo_restante:.0f}s restantes")
        
        print(f"\n{'='*60}")
        print(f"✅ ¡PROCESAMIENTO COMPLETADO!")
        print(f"{'='*60}")
        print(f"\n📊 Estadísticas finales:")
        print(f"   Frames procesados: {frame_count}")
        print(f"   Máximo vehículos simultáneos: {max_vehicles}")
        for cls, name in vehicle_classes.items():
            if stats[cls] > 0:
                print(f"   Máximo de {name}s: {stats[cls]}")
        print(f"\n💾 Video guardado: {output_path}")
        
        return output_path
        
    except KeyboardInterrupt:
        print("\n⚠️ Procesamiento interrumpido por el usuario")
        return None
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return None
    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()

print("✅ Función de detección lista")

✅ Función de detección lista


In [ ]:
# ============================================
# CELDA 8: ¡EJECUTAR DETECCIÓN!
# ============================================
print(f"\n{'='*60}")
print("🚀 COMENZANDO DETECCIÓN DE VEHÍCULOS")
print(f"{'='*60}\n")

output_video = detect_vehicles_macos(
    video_path=VIDEO_PATH,
    output_path=OUTPUT_PATH,
    conf_threshold=CONF_THRESHOLD,
    device=device
)

if output_video:
    print(f"\n{'='*60}")
    print(f"🎉 ¡TODO LISTO!")
    print(f"{'='*60}")
    print(f"\n📹 Tu video procesado está aquí:")
    print(f"   {output_video}")
    # Abrir automáticamente en macOS
    if sys.platform == 'darwin':
        os.system(f'open "{output_video}"')
else:
    print(f"\n❌ Hubo un problema al procesar el video")


🚀 COMENZANDO DETECCIÓN DE VEHÍCULOS


🎬 INICIANDO PROCESAMIENTO

⏳ Procesando 1800 frames...
💡 Esto puede tardar varios minutos

⏳ 1.7% [30/1800] - Vehículos: 14 - ~59s restantes
⏳ 3.3% [60/1800] - Vehículos: 13 - ~58s restantes
⏳ 5.0% [90/1800] - Vehículos: 12 - ~57s restantes
⏳ 6.7% [120/1800] - Vehículos: 14 - ~56s restantes
⏳ 8.3% [150/1800] - Vehículos: 14 - ~55s restantes
⏳ 10.0% [180/1800] - Vehículos: 13 - ~54s restantes
⏳ 11.7% [210/1800] - Vehículos: 12 - ~53s restantes
⏳ 13.3% [240/1800] - Vehículos: 10 - ~52s restantes
⏳ 15.0% [270/1800] - Vehículos: 16 - ~51s restantes
⏳ 16.7% [300/1800] - Vehículos: 14 - ~50s restantes
⏳ 18.3% [330/1800] - Vehículos: 15 - ~49s restantes
⏳ 20.0% [360/1800] - Vehículos: 14 - ~48s restantes
⏳ 21.7% [390/1800] - Vehículos: 15 - ~47s restantes
⏳ 23.3% [420/1800] - Vehículos: 15 - ~46s restantes
⏳ 25.0% [450/1800] - Vehículos: 15 - ~45s restantes
⏳ 26.7% [480/1800] - Vehículos: 14 - ~44s restantes
⏳ 28.3% [510/1800] - Vehículos: 13 - ~43s rest